In [5]:
from sample_test import OperatorTask

obj = OperatorTask('E:/01. 빌리프_컨텐츠아카이빙/Belif')

In [6]:
obj.rename()

Renaming: 2D
Renaming: 3D
Renaming: Collaboration
Renaming: Cutout
Renaming: DM
Renaming: Event
Renaming: Leaflet
Renaming: Logo
Renaming: Model
Renaming: Notice
Renaming: Promotion
Renaming: Sample
Renaming: Video
Renaming: OliveYoung
Renaming: Overseas
Renaming: TaxFree
Renaming: AquaBombJumbo
Renaming: Gratitude
Renaming: Holiday
Renaming: MoisturizingBombJumbo
Renaming: NewYear
Renaming: Summer
Renaming: ThanksGiving
Renaming: OliveYoung
Renaming: Overseas
Renaming: Sephora
Renaming: TaxFree
Renaming: AquaBombJumbo
Renaming: Gratitude
Renaming: Holiday
Renaming: MoisturizingBombJumbo
Renaming: NewYear
Renaming: Summer
Renaming: ThanksGiving
Renaming: OliveYoung
Renaming: Overseas
Renaming: TaxFree
Renaming: AquaBombJumbo
Renaming: Gratitude
Renaming: Holiday
Renaming: MoisturizingBombJumbo
Renaming: NewYear
Renaming: Summer
Renaming: ThanksGiving
Renaming: Kakaotalk
Renaming: OliveYoung
Renaming: Overseas
Renaming: TaxFree
Renaming: Ulta
Renaming: AquaBombJumbo
Renaming: Gratitude


In [ ]:
import pandas as pd
import os, datetime, re

def renameHelp(product_name_path, product_name):
    def extract_volume_info(file_name):
        """ 파일명에서 '100ml', '200g' 같은 정보를 추출 """
        match = re.search(r'(\d+(?:ml|g))', file_name)
        return match.group(1) if match else ""
    
    def extract_country_keyword(file_name):
        """ 파일명에서 사용 국가 키워드 추출 """
        keywords = ['국내', '국내용', '수출용', '해외', '해외용', '중국', '중국용', '국내중국겸용', '일본', '일본용',
                '미국', '북미', '북미용', '북미구주', '유럽', '유럽용', '캐나다', '캐나다용', '미국유럽수출용', '불문',
                '베트남', '베트남용', '동남아', '동남아시아', '동남아시아용']  # 우선순위 높은 순서로 정렬
        
        country_found = [f"_{kw}" for kw in keywords if kw in file_name]
        return ''.join(country_found) if country_found else ''
    
    def extract_detail_keyword(file_name) :
        keywords= ['복수', '단종', 'en'] # 영문 번역본 파악
        for keyword in keywords :
            if keyword in file_name :
                return f"_{keyword}"
        return ''
        
    file_dates = {}  # 중복 방지용 딕셔너리
    
    for root, _, files in os.walk(product_name_path):
        folder_name = os.path.basename(root)            
        for file in sorted(files):
            src_path = os.path.join(root, file)
            mod_time = datetime.datetime.fromtimestamp(os.path.getmtime(src_path)).strftime('%Y%m%d')
            
            # 기존 파일명에서 DOCx_ 문서번호 추출
            doc_match = re.match(r'(DOC\d+)_', file)
            doc_number = doc_match.group(1) if doc_match else ""
            # 부가 정보 추출
            volume_info = extract_volume_info(file)
            volume_suffix = f"_{volume_info}" if volume_info else "" # 용량 정보
            country_suffix = extract_country_keyword(file) # 국가 정보
            dozen_suffix = extract_detail_keyword(file) # 기타(복수/단종) 정보
            ext = os.path.splitext(file)[1] # 확장자
            
            # BasicAsset의 예외 사항
            if product_name == folder_name :
                new_name_key = f"{folder_name}_{mod_time}{country_suffix}{dozen_suffix}{ext}"
                count = file_dates.get(new_name_key, 0) + 1
                file_dates[new_name_key] = count
                new_name = f"{folder_name}_{mod_time}{country_suffix}{dozen_suffix}"
                new_file_name = f"{doc_number}_{new_name}_{count}{ext}" if count > 1 else f"{doc_number}_{new_name}{ext}"
                new_path = os.path.join(root, new_file_name)
                if src_path != new_path:
                  if os.path.exists(new_path) :
                      os.remove(new_path)
                  os.rename(src_path, new_path)
                else :
                  # src_path == new_path인 경우(이름 변경 필요 없음)
                  pass
            else :
                new_name_key = f"{product_name}{volume_suffix}_{folder_name}_{mod_time}{country_suffix}{dozen_suffix}{ext}"
                count = file_dates.get(new_name_key, 0) + 1
                file_dates[new_name_key] = count                 
                new_name = f"{product_name}{volume_suffix}_{folder_name}_{mod_time}{country_suffix}{dozen_suffix}"
                new_file_name = f"{doc_number}_{new_name}_{count}{ext}" if count > 1 else f"{doc_number}_{new_name}{ext}"
                new_path = os.path.join(root, new_file_name)
                # 파일 중복의 경우(DOC_파일명이면 여러 번 이사한 상태)
                if src_path != new_path:
                    if os.path.exists(new_path):
                        os.remove(new_path)
                    os.rename(src_path, new_path)
                else :
                    # src_path == new_path인 경우(이름 변경 필요 없음)
                    pass
                
def rename(base_path) :
    for product_line in os.listdir(base_path):
        product_line_path = os.path.join(base_path, product_line)
        
        # "0_BrandAsset" 폴더 중 Universe 폴더만 건너뛰기
        if product_line == "0_BrandAsset_브랜드자산":
            if os.path.isdir(product_line_path) :
                for brand_asset_type in os.listdir(product_line_path) :
                    if brand_asset_type == 'Universe' :
                        pass
                    else :
                        brand_asset_path = os.path.join(product_line_path, brand_asset_type)
                        renameHelp(brand_asset_path, brand_asset_type)
                        print(f'Renaming: {brand_asset_type}')
        elif product_line == '1_EditionSet_기획세트' :
            if os.path.isdir(product_line_path):
                for year in os.listdir(product_line_path):
                    year_path = os.path.join(product_line_path, year)
                    if os.path.isdir(year_path) :
                        for plan_type in os.listdir(year_path) : # 채널 / 시즌
                            plan_type_path = os.path.join(year_path, plan_type)
                            if os.path.isdir(plan_type_path) :
                                for edition_type in os.listdir(plan_type_path) :
                                    edition_type_path = os.path.join(plan_type_path, edition_type)
                                    renameHelp(edition_type_path, edition_type)
                                    print(f"Renaming: {edition_type}")

        # ProductLine 폴더 내 ProductName 폴더 찾기
        else :
            if os.path.isdir(product_line_path) :
                for product_name in os.listdir(product_line_path):
                    overseas_folderlist = ['1_NorthAmerica_북미', '2_Europe_유럽', '3_UnitedKingdom_영국', '4_OverseasOther_해외기타', 'NorthAmerica_북미', 'Europe_유럽', 'UnitedKingdom_영국', 'OverseasOther_해외기타']           
                    if product_name == '1_DiscontinuedProduct_단종제품' :
                        discontinued_folder_path = os.path.join(product_line_path, product_name)
                        for discontinued_product_name in os.listdir(discontinued_folder_path) :
                            print(f"Renaming: {discontinued_product_name}")
                            discontinued_product_path = os.path.join(discontinued_folder_path, discontinued_product_name)
                            renameHelp(discontinued_product_path, discontinued_product_name)
                    elif product_name in overseas_folderlist :
                        overseas_name_path = os.path.join(product_line_path, product_name) 
                        for overseas_product_name in os.listdir(overseas_name_path) :
                            if overseas_product_name == '1_DiscontinuedProduct_단종제품' :
                                overseas_discontinued_folder_path = os.path.join(overseas_name_path, overseas_product_name)
                                for overseas_discontinued_product_name in os.listdir(overseas_discontinued_folder_path) :
                                    print(f"Renaming: {overseas_discontinued_product_name}")
                                    overseas_discontinued_product_path = os.path.join(overseas_discontinued_folder_path, overseas_discontinued_product_name)
                                    renameHelp(overseas_discontinued_product_path, overseas_discontinued_product_name)
                            else :
                                overseas_product_folder_path = os.path.join(overseas_name_path, overseas_product_name)
                                print(f"Renaming: {overseas_product_name}")
                                renameHelp(overseas_product_folder_path, overseas_product_name)
                    else :
                        product_name_path = os.path.join(product_line_path, product_name)
                        print(f"Renaming: {product_name}")
                        renameHelp(product_name_path, product_name)

In [4]:
base_path = 'E:/01. 빌리프_컨텐츠아카이빙/Belif'

rename(base_path)

Renaming: 2D
Renaming: 3D
Renaming: Collaboration
Renaming: Cutout
Renaming: DM
Renaming: Event
Renaming: Leaflet
Renaming: Logo
Renaming: Model
Renaming: Notice
Renaming: Promotion
Renaming: Sample
Renaming: Video
Renaming: OliveYoung
Renaming: Overseas
Renaming: TaxFree
Renaming: AquaBombJumbo
Renaming: Gratitude
Renaming: Holiday
Renaming: MoisturizingBombJumbo
Renaming: NewYear
Renaming: Summer
Renaming: ThanksGiving
Renaming: OliveYoung
Renaming: Overseas
Renaming: Sephora
Renaming: TaxFree
Renaming: AquaBombJumbo
Renaming: Gratitude
Renaming: Holiday
Renaming: MoisturizingBombJumbo
Renaming: NewYear
Renaming: Summer
Renaming: ThanksGiving
Renaming: OliveYoung
Renaming: Overseas
Renaming: TaxFree
Renaming: AquaBombJumbo
Renaming: Gratitude
Renaming: Holiday
Renaming: MoisturizingBombJumbo
Renaming: NewYear
Renaming: Summer
Renaming: ThanksGiving
Renaming: Kakaotalk
Renaming: OliveYoung
Renaming: Overseas
Renaming: TaxFree
Renaming: Ulta
Renaming: AquaBombJumbo
Renaming: Gratitude
